In [182]:
import pickle as pkl
import datetime

import csv
# Scraping items
from bs4 import BeautifulSoup
import requests

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

import os

from IPython.core.display import display, HTML
import concurrent.futures


import numpy as np
from scipy import stats

import pandas as pd
from subprocess import call
import concurrent.futures
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

#from pymongo import MongoClient

In [128]:
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [129]:
with open('SF_Urls','r') as f:
    urls=pd.read_csv(f)
urls2=[x.strip().strip('\'').strip('\'') for x in list(urls.columns)]

In [130]:
driver = webdriver.Chrome(chromedriver)

In [137]:
tripsf_template='https://trip.sftravel.com/plan-travel/united-states/san-francisco/?page={}'
trip_sf_urls=[tripsf_template.format(i) for i in range(1,89)]

In [141]:
driver.get(trip_sf_urls[0])

In [142]:
soup=BeautifulSoup(driver.page_source,'html.parser')

In [180]:
location_features={}
for url in trip_sf_urls:
    driver.get(url)
    soup=BeautifulSoup(driver.page_source,'html.parser')
    for location in soup.find_all('div',class_='items-item'):
        try:
            features=location.find(class_='items-item-category hidden-xs').text
            name=location.find(class_='items-item-link').text
            location_features[name]=features
        except:
            pass
        

In [268]:
def make_feature_dct(ta_to_sf):
    ta_feature_dct={}
    ta_places=list(ta_to_sf.keys())
    for ta_place in ta_places:
        ta_feature_dct[ta_place]=location_features[ta_to_sf[ta_place]]
    return ta_feature_dct
    

In [270]:
with open('ta_feature_dct.pkl','wb') as f:
    pkl.dump(ta_feature_dct,f)

In [193]:
with open('location_features.pkl','wb') as f:
    pkl.dump(location_features,f)

In [131]:
driver.get(urls2[3])

In [7]:
soup=BeautifulSoup(driver.page_source,'html.parser')


In [6]:
file='data/SF_authors.pkl'
with open(file,'rb') as f:
    names=pkl.load(f)

In [52]:
urls=['https://www.tripadvisor.com/members/'+x for x in names]

In [16]:
driver.get(urls[5])

In [ ]:
https://www.tripadvisor.com/members/LauraT2211

In [17]:
urls[5]

'https://www.tripadvisor.com/members/Mismel2kl'

In [18]:
soup=BeautifulSoup(driver.page_source,'html.parser')

In [111]:
stuff=soup.find_all('li',class_='cs-review')
type(stuff[0])

bs4.element.Tag

In [8]:
attraction_name=stuff[1].find_all(class_='cs-review-location')[0].text
attraction_name

NameError: name 'stuff' is not defined

In [35]:
rating=stuff[0].find_all(class_='cs-review-rating')[0].find('span')
type(rating)

bs4.element.Tag

In [101]:
def get_data(url):
    users={}
    
    name=url[36:]
    driver.get(url)
    soup=BeautifulSoup(driver.page_source,'html.parser')
    reviews=soup.find_all('li',class_='cs-review')
    ranks={}
    for review in reviews:

        try:
            attraction_name=review.find_all(class_='cs-review-location')[0].text
            rating=int([str(tag) for tag in review.find_all()][-3][-10:-9])
        except:
            pass
        #rating=0
        if 'San Francisco:' == attraction_name[:14]:
            ranks[attraction_name]=rating
    users[name]=ranks
    return {name,ranks}
    

In [ ]:
sftrip_data=get_data()

In [89]:
counts = []
with concurrent.futures.ProcessPoolExecutor() as executor:
    well = [executor.submit(get_data, url) for url in urls[:10]]
    counts.append(well)

In [125]:
def scrape_reviews(urls):
    count=0
    tmp={}
    users={}
    for url in urls:
        name=url[36:]
        driver.get(url)
        soup=BeautifulSoup(driver.page_source,'html.parser')
        reviews=soup.find_all('li',class_='cs-review')
        ranks={}
        for review in reviews:

            try:
                attraction_name=review.find_all(class_='cs-review-location')[0].text
                rating=int([str(tag) for tag in review.find_all()][-3][-10:-9])
            except:
                pass
            #rating=0
            if 'San Francisco:' == attraction_name[:14]:
                ranks[attraction_name]=rating
        users[name]=ranks
        tmp[name]=ranks
        count+=1
        if count%100==0:
            with open(f'data/file_{count}.pkl','wb') as f:
                pkl.dump(tmp,f)
            print(count)
            tmp={}
    
    return users
        
    
    

In [255]:
def make_matrix(users,ta_to_sf):
    user_list=list(users.keys())
    # Need full list of SF locations
    places=[]
    for user_key in users.keys():
        for place in users[user_key].keys():
            place=place[15:]
            if place in list(ta_to_sf.keys()):
                places.append(place)
    places=list(set(places))


    # Make array for user1
    matrix=[]
    for u in user_list:
        user_likes=[]
        for p in places:
            if p in users[u].keys():
                user_likes.append(users[u][p])
            else:
                user_likes.append(float('NaN'))
        matrix.append(user_likes)

    return np.array(matrix), user_list, places

In [199]:
results=[]
for k in range(100,10000,100):
    with open(f'data/file_{k}.pkl','rb') as f:
        likes=pkl.load(f)
        results.append(likes)
        
        
res=results[0]
for i in range(1,len(results)):
    res={**res,**results[i]}

In [256]:
matrix, user_list, places = make_matrix(res,ta_to_sf)

In [209]:
places_clean=[x[15:] for x in places]

In [249]:
# want to reduce places_clean list
def make_ta_to_sf(places_clean,sftrip_locations):
    ta_to_sf={}
    for pc in places_clean:
        comp_best=0
        current_closest=None
        for sft in sftrip_locations:
            comp=fuzz.partial_ratio(pc.lower(),sft.lower())
            if comp>comp_best:
                comp_best=comp
                current_closest=sft

        if comp_best>90:
            ta_to_sf[pc]=current_closest
    return ta_to_sf
        

In [242]:
with open('ta_to_sf.pkl','wb') as f:
    pkl.dump(ta_to_sf,f)

In [215]:
sftrip_locations=list(location_features.keys())

In [37]:
with open('likes.pkl','wb') as f:
    pkl.dump(np.array(matrix),f)